# 리뷰 수집 테스트 (가게명 미감지 수정)

이 노트북은 **가게명 미감지 문제**를 해결하기 위해 업데이트되었습니다.

**수정 사항:**
1. **CSV 컬럼명 매핑 수정**: `가게명` -> `restaurant` (CSV 파일 헤더와 일치시킴)
2. **텍스트 추출 방식 개선**: `.text` 대신 `.get_attribute("textContent")` 사용 (숨겨진 텍스트도 추출 가능)
3. **기존 셀렉터 유지**: 사용자가 제공한 셀렉터(`#wrapperDiv > header > div > h2`)를 그대로 사용하되 추출 방식만 변경

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
import time
import datetime
import os
import re

In [27]:
# ==========================
# 환경 설정
# ==========================
INPUT_FILE = "캐치테이블_가게정보.csv"
CUTOFF_DATE = datetime.datetime(2025, 12, 9)
TARGET_INDEX = 0

def get_driver():
    options = Options()
    options.add_argument("--window-size=1600,1000")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=options)
    return driver

def modify_url_for_reviews(url):
    """리뷰 페이지 URL 생성 (sortingFilter=D 추가)"""
    if "/review" in url:
        if "sortingFilter=D" not in url:
             return url + "&sortingFilter=D"
        return url
    if "?" in url:
        base, query = url.split("?", 1)
        if base.endswith("/"): base = base[:-1]
        new_url = f"{base}/review?{query}&sortingFilter=D"
    else:
        if url.endswith("/"): url = url[:-1]
        new_url = f"{url}/review?sortingFilter=D"
    return new_url

def parse_date(date_str):
    today = datetime.datetime.now()
    try:
        if "일 전" in date_str:
            days = int(re.search(r'(\d+)', date_str).group(1))
            return today - datetime.timedelta(days=days)
        elif "시간 전" in date_str or "분 전" in date_str or "방금" in date_str:
            return today
        elif "어제" in date_str:
            return today - datetime.timedelta(days=1)
        else:
            return datetime.datetime.strptime(date_str, "%Y.%m.%d")
    except:
        return today

In [28]:
def scrape_reviews_with_custom_selector(driver, url, restaurant_name):
    # 1. 리뷰 페이지 접속
    review_url = modify_url_for_reviews(url)
    print(f"접속 URL: {review_url}")
    driver.get(review_url)
    time.sleep(5)
    
    collected_reviews = []
    processed_hashes = set()
    
    scrolling = True
    scroll_count = 0
    max_scrolls = 30
    
    # 2. 크롤링 루프
    while scrolling and scroll_count < max_scrolls:
        # 지정된 컨테이너 selector로 카드 찾기 (#main > ... > div)
        try:
            cards = driver.find_elements(By.CSS_SELECTOR, "#main > div.container.gutter-sm > div > div > div > div")
        except:
            cards = []
        
        print(f"스크롤 {scroll_count}: 발견된 카드 {len(cards)}개")
        
        if not cards and scroll_count == 0:
            print("!!! 경고: 카드를 찾지 못했습니다. 셀렉터 확인 필요.")
            driver.save_screenshot("debug_error.png")
        
        found_new = False
        
        for card in cards:
            try:
                # === 상세 데이터 추출 (Relative Selectors) ===
                # 작성자
                try:
                    reviewer = card.find_element(By.CSS_SELECTOR, "article > div.__header > div.__user-info > a > h4 > span").text
                except:
                    reviewer = "Unknown"
                
                # 평점
                try:
                    rating_el = card.find_element(By.CSS_SELECTOR, "article > div.__header > div.__review-meta.__review-meta--with-rating > div > a > div")
                    rating = rating_el.text 
                except:
                    rating = "Unknown"
                    
                # 날짜
                try:
                    date_el = card.find_element(By.CSS_SELECTOR, "article > div.__header > div.__review-meta.__review-meta--with-rating > span")
                    date_text = date_el.text
                except:
                    match = re.search(r'\d{4}\.\d{1,2}\.\d{1,2}|\d+일 전', card.text)
                    if match: date_text = match.group(0)
                    else: date_text = "Unknown"

                # 날짜 파싱 및 검사
                if date_text != "Unknown":
                    review_date = parse_date(date_text)
                    if review_date < CUTOFF_DATE:
                        print(f"   [중단] 기준일 이전: {date_text}")
                        scrolling = False
                        break
                
                # 방문 유형
                try:
                    day_night = card.find_element(By.CSS_SELECTOR, "article > div.__header > div.__review-meta.__review-meta--with-rating > div > p").text
                except:
                    day_night = "Unknown"
                
                # 저장
                msg_hash = hash(f"{reviewer}_{date_text}_{restaurant_name}")
                if msg_hash not in processed_hashes:
                    processed_hashes.add(msg_hash)
                    collected_reviews.append({
                        "restaurant": restaurant_name,
                        "reviewer": reviewer,
                        "review_date": date_text,
                        "reviewer_rating": rating,
                        "day_night": day_night
                    })
                    found_new = True
            except Exception as e:
                continue
        
        if not scrolling:
            break
            
        # === 스크롤 로직 ===
        try:
            if cards:
                last_card = cards[-1]
                actions = ActionChains(driver)
                actions.move_to_element(last_card).perform()
                time.sleep(0.5)
                
                actions.send_keys(Keys.PAGE_DOWN).pause(0.5).send_keys(Keys.PAGE_DOWN).perform()
                time.sleep(1.5)
            else:
                driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
                time.sleep(1)
        except Exception as e:
            driver.find_element(By.TAG_NAME, "body").send_keys(Keys.PAGE_DOWN)
        
        scroll_count += 1
        
    return collected_reviews

In [29]:
# [실행 Block]

if os.path.exists(INPUT_FILE):
    driver = get_driver()
    try:
        df = pd.read_csv(INPUT_FILE)
        if len(df) > TARGET_INDEX:
            target_row = df.iloc[TARGET_INDEX]
            target_url = target_row['URL']
            
            print("1. 식당 메인 정보 확인 중...")
            driver.get(target_url)
            time.sleep(3)
            
            # 가게명 확인 (CSS)
            restaurant_name = "Unknown"
            try:
                # 사용자 제공: /#wrapperDiv > header > div > h2
                # .text가 비어있을 수 있으므로 textContent 사용
                name_element = driver.find_element(By.CSS_SELECTOR, '#wrapperDiv > header > div > h2')
                restaurant_name = name_element.get_attribute("textContent").strip()
                print(f"감지된 가게명(Site): {restaurant_name}")
            except Exception as e:
                print(f"가게명 찾기 실패({e})")
                
            # CSV에서 백업 이름 가져오기 (컬럼명 'restaurant'로 변경)
            if not restaurant_name or restaurant_name == "Unknown":
                if 'restaurant' in df.columns:
                    restaurant_name = target_row['restaurant']
                    print(f"감지된 가게명(CSV): {restaurant_name}")
                elif '가게명' in df.columns:
                     restaurant_name = target_row['가게명']
            
            # 리뷰 개수 확인 (XPath)
            try:
                count_element = driver.find_element(By.XPATH, '//*[@id="wrapperDiv"]/div[1]/div[1]/div[3]/div/span[3]')
                print(f"감지된 리뷰 카운트 텍스트: {count_element.text}")
            except:
                print("리뷰 카운트 요소 찾기 실패")
                
            print(f"\n2. 리뷰 수집 시작: {restaurant_name}")
            reviews = scrape_reviews_with_custom_selector(driver, target_url, restaurant_name)
            
            print(f"\n>>> 최종 수집 결과: {len(reviews)}건")
            if reviews:
                result_df = pd.DataFrame(reviews)
                display(result_df.head())
                result_df.to_csv("test_result_custom.csv", index=False, encoding="utf-8-sig")
    except Exception as e:
        print(f"실행 에러: {e}")
    finally:
        driver.quit()
        print("종료")

1. 식당 메인 정보 확인 중...
감지된 가게명(Site): 
감지된 가게명(CSV): 유용욱 바베큐 연구소
감지된 리뷰 카운트 텍스트: 리뷰 268개

2. 리뷰 수집 시작: 유용욱 바베큐 연구소
접속 URL: https://app.catchtable.co.kr/ct/shop/yyw/review?type=DINING&sortingFilter=D
스크롤 0: 발견된 카드 13개
스크롤 1: 발견된 카드 25개
스크롤 2: 발견된 카드 37개
   [중단] 기준일 이전: 2025.12.08

>>> 최종 수집 결과: 24건


,restaurant,reviewer,review_date,reviewer_rating,day_night
0,유용욱 바베큐 연구소,케미칼,2026.01.13,5.0,저녁
1,유용욱 바베큐 연구소,오니여리,2026.01.12,5.0,점심
2,유용욱 바베큐 연구소,남다른 미식가_15345,2026.01.09,5.0,저녁
3,유용욱 바베큐 연구소,따뜻한 개척가_58539,2026.01.08,5.0,저녁
4,유용욱 바베큐 연구소,sopring,2026.01.08,5.0,저녁


종료
